In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
def get_expiry(f_today):
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
bnf_pandas = pd.read_csv("../data/nifty_wave.csv")

# INDEX = 'MIDCP'
# INDEX = 'BNF'
INDEX = 'NIFTY'
# INDEX = 'FNF'
# INDEX = 'STOCK'
# INDEX = 'SENSEX'

if INDEX == 'BNF':
    lev_ = 5.066
    slippage_ = 0.0001
    lot_size_ = 15
elif INDEX == 'NIFTY' or INDEX == 'FNF':
    lev_ = 6.333
    slippage_ = 0.0002
    lot_size_ = 25
elif INDEX == 'MIDCP':
    lev_ = 6.333
    slippage_ = 0.0005
    lot_size_ = 50
elif INDEX == 'SENSEX':
    lev_ = 7
    slippage_ = 0.0001
    lot_size_ = 10
elif INDEX == 'STOCK':
    lev_ = 5
    slippage_ = 0.001
    lot_size_ = 1
# bnf_pandas = pd.read_csv("../data/finnifty_1hr_tv (2).csv")
# bnf_pandas = pd.read_csv('../data/midcp_1hr_tv (4).csv')
# bnf_pandas = pd.read_csv('../data/finnifty_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/bnf_fut_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')


In [6]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["time"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2015]
bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [7]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

<class 'polars.dataframe.frame.DataFrame'>


In [8]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


In [9]:
# Parameters
ema_short_period = 9
ema_long_period = 21
rsi_period = 14
rsi_short_threshold = 50
rsi_oversold = 30
# stop_loss_pct = 0.5 / 100  # 0.5%
# take_profit_pct = 1.0 / 100  # 1.0%

In [10]:
df = bnf_pandas

In [11]:
# df["ema_short"] = df["close"].ewm(span=ema_short_period, adjust=False).mean()
# df["ema_long"] = df["close"].ewm(span=ema_long_period, adjust=False).mean()

In [12]:
# Calculate RSI
# delta = df["close"].diff()
# gain = delta.where(delta > 0, 0).rolling(window=rsi_period).mean()
# loss = -delta.where(delta < 0, 0).rolling(window=rsi_period).mean()
# rs = gain / loss
# df["rsi"] = 100 - (100 / (1 + rs))

In [13]:
def short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold):

    df["ema_short"] = df["close"].ewm(span=ema_short_period, adjust=False).mean()
    df["ema_long"] = df["close"].ewm(span=ema_long_period, adjust=False).mean()

    # Calculate RSI
    delta = df["close"].diff()
    gain = delta.where(delta > 0, 0).rolling(window=rsi_period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=rsi_period).mean()
    rs = gain / loss
    df["rsi"] = 100 - (100 / (1 + rs))
    
    # Initialize variables
    position = None  # None, 'short'
    tradebook = []
    signal_time = None
    signal_price = None
    entry_time = None
    entry_price = None
    initial_sl = None
    
    # Iterate through rows to generate signals
    for i in range(len(df)):
        row = df.iloc[i]
    
        # Check for short signal condition
        if position is None:
            if row["ema_short"] < row["ema_long"] and row["rsi"] < rsi_short_threshold:
                if signal_price is None:  # New signal
                    signal_time = row["datetime"]
                    signal_price = row["low"]
                else:
                    if row['low'] >= signal_price:
                        # Update Signal Price to Better Candle low
                        signal_time = row["datetime"]
                        signal_price = row["low"]
    
            if signal_price is not None:
                if row["low"] < signal_price and row['high'] >= signal_price:  # Signal broken
                    entry_time = row["datetime"]
                    entry_price = signal_price
                    initial_sl = max(df["high"][max(0, i - 3):i].max(), row["high"])
                    position = "short"
        
                    # Add trade to tradebook
                    tradebook.append({
                        "Signal Time": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": None,
                        "Exit Price": None,
                        "Points Captured": None
                    })
                    signal_time, signal_price = None, None  # Reset signal
                else:
                    signal_price = None
                    if row["ema_short"] < row["ema_long"] and row["rsi"] < rsi_short_threshold:
                        if signal_price is None:  # New signal
                            signal_time = row["datetime"]
                            signal_price = row["low"]
                        else:
                            if row['low'] >= signal_price:
                                # Update Signal Price to Better Candle low
                                signal_time = row["datetime"]
                                signal_price = row["low"]
    
        # Exit logic for short position
        if position is not None:
            if row['close'] >= initial_sl:
                exit_time = row["datetime"]
                exit_price = row['close']
                points_captured = entry_price - exit_price
    
                # Update tradebook
                tradebook[-1]["Exit Time"] = exit_time
                tradebook[-1]["Exit Price"] = exit_price
                tradebook[-1]["Points Captured"] = points_captured
                tradebook[-1]["Remark"] = 'Initial SL Hit'
    
                # Reset position
                position = None
                entry_time, entry_price, initial_sl = None, None, None
            elif row["rsi"] > rsi_oversold:
                exit_time = row["datetime"]
                exit_price = row["close"]
                points_captured = entry_price - exit_price
    
                # Update tradebook
                tradebook[-1]["Exit Time"] = exit_time
                tradebook[-1]["Exit Price"] = exit_price
                tradebook[-1]["Points Captured"] = points_captured
                tradebook[-1]["Remark"] = 'RSI Oversold'
    
                # Reset position
                position = None
                entry_time, entry_price, initial_sl = None, None, None
    
    # Convert tradebook to a DataFrame
    tradebook_df = pd.DataFrame(tradebook)
    return tradebook_df
    # Output
    # print(tradebook_df)
    
    # Optional: Save to CSV
    # tradebook_df.to_csv("tradebook.csv", index=False)


In [15]:
ema_short_period = 7
ema_long_period = 9
rsi_period = 6
rsi_short_threshold = 70
rsi_oversold = 20
tradebook_df = short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold)
# tradebook_df

In [16]:
tradebook_df['Qty'] = 5 * 10000000 / tradebook_df['Entry Price'] #5x Lev, 1Cr portfolio
tradebook_df['PnL'] = tradebook_df['Points Captured'] * tradebook_df['Qty']
tradebook_df['PnL w cs'] = (tradebook_df['Points Captured'] - slippage_*(tradebook_df['Entry Price'] + tradebook_df['Exit Price'])) * tradebook_df['Qty']
tradebook_df['ROI%'] = tradebook_df['PnL w cs'] * 100 / 10000000

In [17]:
tradebook_df['Trade Year'] = tradebook_df['Entry Time'].dt.year

In [18]:
tradebook_df['ROI%'].sum()

131.61383327434945

In [19]:
tradebook_df[tradebook_df['Trade Year'] >= 2024]

,Signal Time,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Points Captured,Remark,Qty,PnL,PnL w cs,ROI%,Trade Year
1476,2024-01-02 09:15:00,2024-01-02 10:15:00,21626.5000,21834.3500,2024-01-02 10:15:00,21588.4000,38.1000,RSI Oversold,2311.9784,88086.3755,68103.9928,0.6810,2024
1477,2024-01-02 15:15:00,2024-01-03 09:15:00,21657.0000,21692.7000,2024-01-03 09:15:00,21586.4000,70.6000,RSI Oversold,2308.7224,162995.7981,143028.3973,1.4303,2024
1478,2024-01-03 11:15:00,2024-01-03 12:15:00,21548.1500,21677.0000,2024-01-04 09:15:00,21631.5000,-83.3500,RSI Oversold,2320.3848,-193404.0741,-213442.7549,-2.1344,2024
1479,2024-01-08 12:15:00,2024-01-08 13:15:00,21566.0500,21724.5000,2024-01-09 09:15:00,21657.3500,-91.3000,RSI Oversold,2318.4589,-211675.2952,-231717.6303,-2.3172,2024
1480,2024-01-09 14:15:00,2024-01-09 15:15:00,21533.1000,21724.4500,2024-01-09 15:15:00,21551.9500,-18.8500,RSI Oversold,2322.0066,-43769.8241,-63778.5781,-0.6378,2024
1481,2024-01-10 10:15:00,2024-01-10 11:15:00,21512.1000,21573.9000,2024-01-10 13:15:00,21531.9000,-19.8000,RSI Oversold,2324.2733,-46020.6117,-66029.8158,-0.6603,2024
1482,2024-01-17 10:15:00,2024-01-17 11:15:00,21727.1500,22036.6000,2024-01-18 10:15:00,21484.2500,242.9000,RSI Oversold,2301.2682,558978.0528,539089.8484,5.3909,2024
1483,2024-01-18 11:15:00,2024-01-18 12:15:00,21444.9500,21539.4000,2024-01-18 12:15:00,21452.5000,-7.5500,RSI Oversold,2331.5513,-17603.2119,-37606.7326,-0.3761,2024
1484,2024-01-18 13:15:00,2024-01-18 14:15:00,21432.0500,21539.4000,2024-01-18 14:15:00,21458.3000,-26.2500,RSI Oversold,2332.9546,-61240.0587,-81252.3067,-0.8125,2024
1485,2024-01-19 10:15:00,2024-01-19 11:15:00,21577.1000,21670.6000,2024-01-19 11:15:00,21588.4000,-11.3000,RSI Oversold,2317.2716,-26185.1685,-46190.4056,-0.4619,2024


In [20]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2015, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2015, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [21]:
variation = f'...{rsi_oversold}'
stats = generate_stats(tradebook_df, variation)

In [22]:
stats_dictionary = {}
for overall_roi_dd_ratio, stats_df in stats.items():
    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
        print(stats_df.to_string())
        stats_dictionary[overall_roi_dd_ratio] = stats_df

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2015      37.8015          169  43.7870                2.0112             -1.1687     -14.8190       2.5509     ...20
2016      16.5132          187  44.9198                1.4280             -1.0042     -21.1169       0.7820     ...20
2017     -25.4440          148  36.4865                0.8922             -0.7832     -31.7703      -0.8009     ...20
2018      -1.6947          152  38.8158                1.8910             -1.2179     -25.4829      -0.0665     ...20
2019      -9.0851          176  40.9091                1.6798             -1.2503     -30.3120      -0.2997     ...20
2020      59.0958          145  40.0000                3.3261             -1.5381     -22.8653       2.5845     ...20
2021      22.5311          145  45.5172                1.7751             -1.1978     -18.3973       1.2247     ...20
2022      13.0168          187  47.0588                1

In [37]:
stats_dictionary = {}

for ema_short_period in range(5, 16, 2):
    for ema_long_period in range(5, 24, 2):
        for rsi_period in range(2, 15, 2):
            for rsi_short_threshold in range(20, 81, 10):
                for rsi_oversold in range(20, 61, 10):
                    if (ema_short_period < ema_long_period) and (rsi_oversold <= rsi_short_threshold):
                        tradebook_df = short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold)
                        tradebook_df['Qty'] = 5 * 10000000 / tradebook_df['Entry Price'] #5x Lev, 1Cr portfolio
                        tradebook_df['PnL'] = tradebook_df['Points Captured'] * tradebook_df['Qty']
                        tradebook_df['ROI%'] = tradebook_df['PnL'] * 100 / 10000000
                        tradebook_df['Trade Year'] = tradebook_df['Entry Time'].dt.year
                        variation = f'{ema_short_period}, {ema_long_period}, {rsi_period}, {rsi_short_threshold}, {rsi_oversold} -> {round(tradebook_df["ROI%"].sum())}%'
                        print(variation)
                        new_tb = tradebook_df
                        if len(new_tb) > 1:
                            new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
                            stats = generate_stats(new_tb, variation)
                            for overall_roi_dd_ratio, stats_df in stats.items():
                                if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 9:
                                    print(stats_df.to_string())
                                    stats_dictionary[overall_roi_dd_ratio] = stats_df

5, 7, 2, 20, 20 -> 190%
5, 7, 2, 30, 20 -> 206%
5, 7, 2, 30, 30 -> 224%
5, 7, 2, 40, 20 -> 256%
5, 7, 2, 40, 30 -> 258%
5, 7, 2, 40, 40 -> 247%
5, 7, 2, 50, 20 -> 235%
5, 7, 2, 50, 30 -> 254%
5, 7, 2, 50, 40 -> 252%
5, 7, 2, 50, 50 -> 204%
5, 7, 2, 60, 20 -> 246%
5, 7, 2, 60, 30 -> 250%
5, 7, 2, 60, 40 -> 241%
5, 7, 2, 60, 50 -> 193%
5, 7, 2, 60, 60 -> 192%
5, 7, 2, 70, 20 -> 277%
5, 7, 2, 70, 30 -> 282%
5, 7, 2, 70, 40 -> 257%
5, 7, 2, 70, 50 -> 202%
5, 7, 2, 70, 60 -> 205%
5, 7, 2, 80, 20 -> 278%
5, 7, 2, 80, 30 -> 297%
5, 7, 2, 80, 40 -> 267%
5, 7, 2, 80, 50 -> 223%
5, 7, 2, 80, 60 -> 215%
5, 7, 4, 20, 20 -> 202%
5, 7, 4, 30, 20 -> 239%
5, 7, 4, 30, 30 -> 240%
5, 7, 4, 40, 20 -> 272%
5, 7, 4, 40, 30 -> 268%
5, 7, 4, 40, 40 -> 274%
5, 7, 4, 50, 20 -> 311%
5, 7, 4, 50, 30 -> 288%
5, 7, 4, 50, 40 -> 292%
5, 7, 4, 50, 50 -> 280%
5, 7, 4, 60, 20 -> 356%
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                Variation
201

In [38]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
sorted_stats

{23.204395852905588:         Total ROI Total Trades Win Rate Avg Profit% per Trade  \
 2015      71.5872          169  52.0710                1.8766   
 2016      53.9024          187  51.3369                1.4375   
 2017       4.1551          148  44.5946                0.9058   
 2018      28.6996          152  48.6842                1.6878   
 2019      26.1096          176  47.7273                1.6233   
 2020      88.0781          145  43.4483                3.2517   
 2021      51.5208          145  48.9655                1.8438   
 2022      50.4067          187  53.4759                1.5671   
 2023      32.2288          167  52.0958                1.2144   
 2024      48.8343          144  47.9167                1.7618   
 Overall  455.5227         1620  49.2593                1.6840   
 
         Avg Loss% per Trade Max Drawdown ROI/DD Ratio                Variation  
 2015                -1.1549     -12.0810       5.9256  7, 9, 6, 70, 20 -> 456%  
 2016                -